In [15]:
# -------------------------------
#Importing Dependencies
# -------------------------------
import csv
import os
# Organisation Libraries 
import pandas as pd
import numpy as np

# SQL Libraries
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

import time
from getpass import getpass

# ML Libraryies
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

In [17]:
# Using psycopg2 to pull from the NBA_Analysis DB
password = getpass()
cn = psycopg2.connect(host = "127.0.0.1", port = "5432", database = "NBA_Analysis", user = "postgres", password = password)
cur = cn.cursor()


#Querying Bigman Data
cur.execute("""
SELECT main.id, main.player, main.u_player, misc.stl, misc.drb, misc.blk
FROM main  
LEFT JOIN misc
ON main.id = misc.id;
""")
d_results = cur.fetchall()

# Getting the column headers for Bigman Table
cur.execute("""
SELECT main.id, main.player, main.u_player, misc.stl, misc.drb, misc.blk
FROM main  
LEFT JOIN misc
ON main.id = misc.id LIMIT 0;
""")
d_colnames = [desc[0] for desc in cur.description]

# cur.execute("Select * FROM misc")
# query_results2 = cur.fetchall()


 ········


In [19]:
# Assigning SQL data to df's
d_df = pd.DataFrame(d_results)
d_df.columns = d_colnames
d_df.head()


,id,player,u_player,stl,drb,blk
0,1,Álex Abrines,Álex Abrines 2016-17,0.5,1.0,0.1
1,2,Quincy Acy,Quincy Acy 2016-17,0.4,2.5,0.4
2,3,Steven Adams,Steven Adams 2016-17,1.1,4.2,1.0
3,4,Arron Afflalo,Arron Afflalo 2016-17,0.3,1.9,0.1
4,5,Alexis Ajinça,Alexis Ajinça 2016-17,0.5,3.4,0.6


In [35]:
# Creating a defensive player dfd
stl = d_df.groupby("player").mean()['stl']
drb = d_df.groupby("player").mean()['drb']
blk = d_df.groupby("player").mean()['blk']

d_clean_df = pd.DataFrame({
    "steals": stl,
    "def_reb": drb,
    "blocks": blk
})

#it looks like we have 69 duplictes, i looked through the list here and 
# it seems liek they are all scrubs. I vote for dropping the duplicates.
dict(d_clean_df.duplicated().sort_values(ascending = False).head(69))

{'Jacob Pullen': True,
 'Cassius Stanley': True,
 'Quincy Pondexter': True,
 'Tariq Owens': True,
 'Tyler Cook': True,
 'Tyler Davis': True,
 'George King': True,
 'Gary Neal': True,
 'Tahjere McCall': True,
 'Cassius Winston': True,
 'Mike Tobey': True,
 'Tyrell Terry': True,
 'Kendrick Perkins': True,
 'Keljin Blevins': True,
 'Markel Brown': True,
 'Steve Novak': True,
 'Vic Law': True,
 'Vince Edwards': True,
 'Ty-Shon Alexander': True,
 'Mike Dunleavy': True,
 'Jonah Bolden': True,
 'DeVaughn Akoon-Purcell': True,
 'Kyle Wiltjer': True,
 'David Stockton': True,
 'Thomas Welsh': True,
 'Richard Jefferson': True,
 'Matt Williams': True,
 'Mfiondu Kabengele': True,
 'Greg Whittington': True,
 'Reggie Hearn': True,
 'Mason Jones': True,
 'Kyle Alexander': True,
 'Larry Drew II': True,
 'Rawle Alkins': True,
 'Trevon Duval': True,
 'Jeremy Evans': True,
 'Randy Foye': True,
 'Trey McKinney-Jones': True,
 'Vince Hunter': True,
 'Luis Montero': True,
 'Jordan Sibert': True,
 'Zach Norvel

In [ ]:
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df1)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)